In [1]:
try:
    %tensorflow_version 2.x
except Exception:
    pass

In [2]:
import json
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import datetime
import pickle

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
#tf.config.gpu.set_per_process_memory_fraction(0.75)

In [4]:
# tf.config.experimental.list_physical_devices('GPU')

In [5]:
# physical_devices = tf.config.list_physical_devices('GPU') 
# try: 
#   tf.config.set_logical_device_configuration( 
#     physical_devices[0], 
#     [tf.config.LogicalDeviceConfiguration(memory_limit=5000), 
#     ]) 
 
# #   logical_devices = tf.config.list_logical_devices('GPU') 
# #   assert len(logical_devices) == len(physical_devices) + 1 
 
# #   tf.config.set_logical_device_configuration( 
# #     physical_devices[0], 
# #     [tf.config.LogicalDeviceConfiguration(memory_limit=10), 
# #      tf.config.LogicalDeviceConfiguration(memory_limit=10)]) 
# except Exception as e: 
#   print("Invalid device or cannot modify logical devices once initialized.") 
#   print(e) 


In [6]:
# try: 
#   tf.config.experimental.set_memory_growth(physical_devices[0], True) 
#   assert tf.config.experimental.get_memory_growth(physical_devices[0]) 
# except: 
#   # Invalid device or cannot modify virtual devices once initialized. 
#   pass 


In [7]:
# from __future__ import absolute_import, division, print_function, unicode_literals


# print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


sample code to show tokenization of sentences


In [8]:

# sentences = [
#     'i love my dog',
#     'I, love my cat',
#     'You love my dog!',#focus on how tokenizer didnt create a new word for dog!
#     'do you think my dog is amazing?'# adding new words
    
# ]

# tokenizer = Tokenizer(num_words = 100,oov_token = "<OOV>")# adding out of vocabulary to ensure we do not lose the len of sentence for the words it hasnt seen before

# tokenizer.fit_on_texts(sentences)
# word_index = tokenizer.word_index
# print(word_index)

In [9]:
# sequences = tokenizer.texts_to_sequences(sentences)# simply shows texts interpreted as numbers generated by tokenizer
# #simple tensor can not handle unequal length sentences
# #teherfor we'll have to pad the sentences to ensure an equal length
# #or USE RAGGED TENSOR
# padded = pad_sequences(sequences,maxlen=5)#if len is more than this specify how to truncate i.e. pre or post
# #padding can be done both pre and post by  usin padding ='post' or 'pre' default is pre
# #also truncatin ='post' or 'pre' can be used default is pre with maxlen

# print('\nWord_index =',word_index)
# print("\nSequences =",sequences)
# print("\nPadded Sequences:",padded)


In [10]:
# test_data = [
#     'i really love my dog',
#     'my dog loves my mantee'
# ]
# test_seq= tokenizer.texts_to_sequences(test_data)
# padded_test_seq = pad_sequences(test_seq,maxlen=10)
# print (padded_test_seq)

# Sentiment (sarcasm) analysis on news head lines

In [11]:
#importing data

!wget --no-check-certificate\
   https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json \
   -O /tmp/sarcasm.json

--2020-03-28 21:34:24--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 2404:6800:4002:80a::2010, 172.217.26.240
Connecting to storage.googleapis.com (storage.googleapis.com)|2404:6800:4002:80a::2010|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘/tmp/sarcasm.json’

/tmp/sarcasm.json    49%[========>           ]   2.68M  62.4KB/s    eta 29s    ^C


In [12]:
vocab_size =10000
embedding_dim = 16
max_length = 100
trunc_type = 'post'
padding_type='post'
oov_token = "<OOV>"
training_size = 20000


In [13]:
with open("/tmp/sarcasm.json",'r') as f:
    datastore = json.load(f)
    #simply opening json file and storing it in python list format
    
sentences =[]
labels =[]

for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])
    
    
    

JSONDecodeError: Unterminated string starting at: line 13321 column 2 (char 2799176)

In [ ]:
d= {'sentences':sentences,'labels':labels}
df = pd.DataFrame(d)
 
df['labels'].value_counts()
df.head()
is_sarcastic = df[df['labels']==1]
not_sarcastic = df[df['labels']==0]
is_sarcastic.shape,not_sarcastic.shape

not_sarcastic= not_sarcastic.sample(is_sarcastic.shape[0])

df =is_sarcastic.append(not_sarcastic,ignore_index=True)
df =df.sample(frac=1).reset_index(drop=True)

#The frac keyword argument specifies the fraction of rows to return in the random sample, so frac=1 means return all rows (in random order).
#Here, specifying drop=True prevents .reset_index from creating a column containing the old index entries.

In [ ]:
training_data = df[0:training_size]
testing_data = df[training_size:]

In [ ]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_token)
#initailises tokeniser with vocab size and oov token
tokenizer.fit_on_texts(training_data['sentences'])

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_data['sentences'])
training_padded = pad_sequences(training_sequences, maxlen= max_length,padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_data['sentences'])
testing_padded = pad_sequences(testing_sequences, maxlen= max_length,padding=padding_type, truncating=trunc_type)


import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
#need this block to get it to work with tensorflow 2.x

training_padded =np.array(training_padded)
testing_padded =np.array(testing_padded)
training_labels =np.array(training_data['labels'])
testing_labels =np.array(testing_data['labels'])

In [ ]:
#model

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_length),
    #Turns positive integers (indexes) into dense vectors of fixed size. eg. [[4], [20]] -> [[0.25, 0.1], [0.6, -0.2]]
    #use some kind of mapping
    #https://keras.io/layers/embeddings/     for reference
    #Input shape : 2D tensor with shape: (batch_size, sequence_length).
    #Output shape : 3D tensor with shape: (batch_size, sequence_length, output_dim).
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')

])

model.compile(loss='binary_crossentropy',optimizer = 'adam',metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
path = "/home/deity/Documents/sentiment_analysis/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callbacks = tf.keras.callbacks.TensorBoard(log_dir=path,profile_batch=0)
                                                       #histogram_freq=1
                                                      #write_graph=True,
                                                       #write_images=True,
                                                       #update_freq='epoch',
                                                       #embeddings_freq=5,
                                                     # )

In [ ]:
num_epochs = 25
history = model.fit(training_padded,training_labels,epochs = num_epochs,
                   validation_data=(testing_padded,testing_labels),verbose =2,
                   callbacks=[tensorboard_callbacks])

In [ ]:
filename = 'Sentimentanlysismodel'
pickle.dump(model,open(filename,'wb'))

In [ ]:
model.save('/home/deity/Documents/sentiment_analysis/mymodel')

In [ ]:
def plot_graphs(history,string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.xlabel("epochs")
    plt.ylabel(string)
    plt.legend([string,'val_'+string])
    plt.show()
    
plot_graphs(history,"accuracy")
plot_graphs(history,"loss")
    

In [ ]:
reverse_word_index = dict([(value,key) for (key,value) in word_index.items()])

def decode_sentence(text):
    return ' '.join([reverse_word_index.get(i,'?') for i in text])

print(decode_sentence(training_padded[2]))
print(df['sentences'][2])
print(df['labels'][2])

In [ ]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) 
#shape : (vocab_size, embedding_dim)

In [14]:
%load_ext tensorboard

In [15]:
%tensorboard --logdir logs/fit

In [ ]:
sen =["granny starting to fear spiders in the garden might be real", "game of thrones season finale showing this sunday night"]
sentence = sen
print (type(sentence))
sequences = Tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(model.predict(padded))